# ML/ Image Quality Detector
> part 5
Assembly the entire model (NN)

### LET's ASSEMBLE the entire model (NN)

## First, we will modify our CNN layer so it:

1. has 32 kernels with the dimension 20x20 pixels
2. we will configure the kernels to "walk" from 20 pixels(stride)
3. the layer will output 32 images of 25 x 25 "PIXELS", each pixel will coresponde to a "sector" in the original image
<img src="./imagesCNN/img5.png" style="width:900px;height 900px"/>

## Secound, we will:
1. "flaten" the CNN output so it fits into a Linear Neuronal Layers, so 32 images with 32 images with 25 will flaten into a vector of 1 x 20000 values
2. at the output of the linear layer we will have a vector of 2
<img src="./imagesCNN/img6.png" style="width:900px;height 900px"/>

In [130]:
import os

from PIL import Image,ImageOps
import matplotlib.pyplot as plt
import numpy as np 

import torch, torch.nn as nn, torch.optim as optim


In [131]:
# Image Helpers 
def loadImages(folder,number=50):
    images = []
    count = 0
    file_names = os.listdir(folder)
    for file_name in file_names:
            img = Image.open(folder+"/"+file_name)
            img = ImageOps.grayscale(img)
            img_matrix= np.array(img)
            images.append(img_matrix)
            # print(file_name)
            count += 1
            if count == number:
                  break

            
    return images

def imageToTensor(img):
      img_tensor = torch.from_numpy(img).type(torch.FloatTensor).view(1,500,500)
      return img_tensor

In [132]:
# Prep The Model     -->  create multi layer of neurons NN
model = nn.Sequential(
    #                            v--nn.layers      V-- pixels taken
    nn.Conv2d(in_channels=1,out_channels=32,kernel_size=(20,20), stride = (20,20)), # layer 0 <- Neurons
    nn.Flatten(start_dim=0), # No have neurons  transform function
    nn.Linear(in_features=20000, out_features=2),                                    # layer 1 <- Neurons
)

In [138]:
# PREPARING DATASET
original_images  = loadImages("images/original")
# blurred_images   = loadImages("images/blurred")   
lowq_images      = loadImages("images/lowq")      
# sharpened_images = loadImages("images/sharpened")

print(f"loaded{len(original_images)+ len(lowq_images)} images")


loaded100 images


In [139]:
## Preparing Data sets

x = original_images + lowq_images # 100 [50 - good, 50 - bad]
#      V --- GOOD                    V-- BAD 
y = [[0,1] for _ in range(50)] + [[1,0] for _ in range(50)]

In [135]:
# pass one image trough the CNN
img_tensor = imageToTensor(original_images[49])
otuput= model(img_tensor)
print(otuput.shape)
print(otuput)


torch.Size([2])
tensor([ 6.8441, 29.3239], grad_fn=<AddBackward0>)


In [ ]:
## Training setup
from turtle import mode

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(),lr=0.00001)

for epoch in range(1000):
    for i in range(len(x)):
        x_tensor= imageToTensor(x[i])
        y_tensor = torch.FloatTensor(y[i])

        optimizer.zero_grad()
        y_predicted = model(x_tensor)

        loss = criterion(y_predicted, y_tensor)

        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f"epoch = {epoch:5},loss= {loss:10.8f}")

In [140]:
## Let's test it
correct = 0 
for i in range(len(x)):
    x_tensor= imageToTensor(x[i])
    y_predicted = model(x_tensor)
    y_tensor = torch.FloatTensor(y[i])
    if(torch.argmax(y_predicted)) == torch.argmax(y_tensor):
        correct += 1

        
accuracy = (correct / len(x))* 100       
print(f"predicted correctly {accuracy}%")


predicted correctly 52.0%


In [ ]:
## Check what the filters "learned"
for f in  model[0].weight:
    plt.figure()
    plt.imshow(f.detach().numpy()[0])
    

## Conclusion 
- how to optimize
1. expend the dataset up to 1000 images
2. increase the filters up to 64
3. increase the number of epochs 10000
4. optimize on GPU